This script shows how to check if 2 rasters are identical.


in your python conda environment. (python3.8)

conda install -c conda-forge gdal



In [35]:
from osgeo import gdal
import numpy as np  #yup, numpy AGAIN!

In [36]:

#we have a few rasters:

# 1_almost.tif  1_other.tif  1.tif  2.tif  (in 'rasters' subfolder)

# we want to check which rasters are equal to 1.tif

In [37]:
#make a function so we can re-use code
#will return True if they are equal, False if they are not.

def check_raster_equal(raster1_name, raster2_name, debug=False):

    #open both
    
    ds1 = gdal.Open(raster1_name)
    if ds1 is None:
        print('bad raster1_name, looks like file does not exist?')
        return False
    
    ds2 = gdal.Open(raster2_name)
    if ds2 is None:
        print('bad raster2_name, looks like file does not exist?')
        return False
    
    #check they have the same number of bands:
    if ds1.RasterCount != ds2.RasterCount:
        if debug:
            print('not the same number of bands')
        return False
    
    #check they have the same geotransform
    if ds1.GetGeoTransform() != ds2.GetGeoTransform():
        if debug:
            print('not the same geotransform')
        return False

    #check they have the same projection
    if ds1.GetProjection() != ds2.GetProjection():
        if debug:
            print('not the same projection')
        return False
    
    if ds1.RasterXSize != ds2.RasterXSize or ds1.RasterYSize != ds2.RasterYSize:
        if debug:
            print('data not the same shape')
        return False
    
    #save this one for last because part of it involves loading the data
    #now check the data. raster bands are 1-based.
    #for each band, check the data
    for i in range(1, ds1.RasterCount + 1):
        
        #check band nodata values
        if ds1.GetRasterBand(1).GetNoDataValue() != ds2.GetRasterBand(1).GetNoDataValue():
            if debug:
                print(f'band {i} nodata values not the same')
            return False
        
        data1 = ds1.GetRasterBand(1).ReadAsArray() #returns numpy array
        data2 = ds2.GetRasterBand(1).ReadAsArray() #returns numpy array

        if not np.allclose(data1, data2):
            if debug:
                print('data differs')
            return False
            
    
    return True



In [38]:

print(check_raster_equal('./rasters/1.tif', './rasters/2.tif', debug=True))
print()

print(check_raster_equal('./rasters/1.tif', './rasters/1_other.tif', debug=True))
print()


print(check_raster_equal('./rasters/1.tif', './rasters/1_almost.tif', debug=True))
print()



not the same geotransform
False

True

data differs
False



In [39]:
#and without any debug messages
print(check_raster_equal('./rasters/1.tif', './rasters/2.tif'))
print(check_raster_equal('./rasters/1.tif', './rasters/1_other.tif'))
print(check_raster_equal('./rasters/1.tif', './rasters/1_almost.tif'))


False
True
False
